In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from iob_ia.utils import io_utils, measure, visualise

C:\Users\Microscopy\.conda\envs\iob-ia\lib\site-packages\cupy\_environment.py:217: UserWarning: CUDA path could not be detected. Set CUDA_PATH environment variable if CuPy fails to load.
  warnings.warn(


Specify paths

## Segment
already segmented

--> load segmentations (10x - crops)

In [4]:
# paths
path_dapi = 'G:\\20241120_IOB_Magdalena\\20250520_TestClearing\\Organoid 1\\Crops\\20250520_10x_Nq145um-C1-crop.tif'
path_gfp = 'G:\\20241120_IOB_Magdalena\\20250520_TestClearing\\Organoid 1\\Crops\\20250520_10x_Nq145um-C2-crop.tif'
path_cy5 = 'G:\\20241120_IOB_Magdalena\\20250520_TestClearing\\Organoid 1\\Crops\\20250520_10x_Nq145um-C3-crop.tif'
path_gfp_mask = 'G:\\20241120_IOB_Magdalena\\20250520_TestClearing\\Organoid 1\\Crops\\20250520_10x_Nq145um-C2-crop__cp-mask.tif'
path_cy5_mask = 'G:\\20241120_IOB_Magdalena\\20250520_TestClearing\\Organoid 1\\Crops\\20250520_10x_Nq145um-C3-crop__cp-mask.tif'

# load images and masks as BioImage objects
dapi = io_utils.read_image(path_dapi)
gfp = io_utils.read_image(path_gfp)
cy5 = io_utils.read_image(path_cy5)
gfp_mask = io_utils.read_labels(path_gfp_mask)
cy5_mask = io_utils.read_labels(path_cy5_mask)


Probably need to filter the masks:
- size (min)
- and mean intensity, maybe go for something automatic (mean of mean-intensities - 10% -> check what i have done so far)

## Visualise

In [5]:
visualise.add_image(img=dapi.get_data(), name='dapi', colormap='blue') # scale = dapi.physical_pixel_sizes
visualise.add_image(img=gfp.get_data(), name='gfp', colormap='green')
visualise.add_image(img=cy5.get_data(), name='640', colormap='red')
visualise.add_labels(img=gfp_mask, name='gfp_mask') #, scale=dapi.physical_pixel_sizes)
visualise.add_labels(img=cy5_mask, name='cy5_mask')

In [6]:
print(dapi.physical_pixel_sizes)

PhysicalPixelSizes(Z=1.45, Y=0.6500002275000796, X=0.6500002275000796)


### --> try to find double positively segmented nuclei

I think for an initial start it would be good to go two ways
1. check segmentation overlap (min 50%) red cells that are also green and opposite
   - This needs a new function...
3. measure intensities per channel cell masks
   - I probably did already this function

In [6]:
cy5_pos_gfp_mask = measure.overlapping_labels(cy5_mask, gfp_mask)
visualise.add_labels(img=cy5_pos_gfp_mask, name="gfp_positive_Cy5-Objects")

In [18]:
import numpy as np
import pandas as pd

In [17]:
np.unique(cy5_pos_gfp_mask)

array([  0,   7,  19,  27,  38,  42,  57,  67,  95, 124, 130, 137, 138,
       143, 154, 221, 238, 272, 273, 291, 292, 295, 301, 320, 355, 362,
       409, 428, 429, 432, 435, 447, 455, 475, 492, 505, 508])

In [21]:
a = measure.intensity_and_presence(cy5_mask, gfp.get_data(), cy5_pos_gfp_mask)

In [20]:
print(cy5_mask.shape, gfp.get_data().shape, cy5_pos_gfp_mask.shape)

(427, 350, 350) (427, 350, 350) (427, 350, 350)


In [25]:
for i, r in pd.DataFrame.from_dict(a).iterrows():
    print(r["label"], r["intensity_mean"], r["positive"], 'i =', i)

1.0 154.5894495412844 0.0
2.0 128.5618153364632 0.0
3.0 121.43373493975903 0.0
4.0 79.41753472222223 0.0
5.0 101.0310606060606 0.0
6.0 91.71077762619372 0.0
7.0 696.688524590164 1.0
8.0 90.23624595469256 0.0
9.0 101.70222222222222 0.0
10.0 158.0743455497382 0.0
11.0 128.0 0.0
12.0 113.89311740890689 0.0
13.0 490.47947454844007 0.0
14.0 121.38978494623656 0.0
15.0 157.0181329751511 0.0
16.0 93.97218543046358 0.0
17.0 143.5635062611807 0.0
18.0 159.333479020979 0.0
19.0 428.7590987868284 1.0
20.0 168.33604887983708 0.0
21.0 108.65930599369085 0.0
22.0 199.53429672447012 0.0
23.0 188.7 0.0
24.0 109.24167872648336 0.0
25.0 182.84655172413792 0.0
26.0 95.79263711495116 0.0
27.0 1000.85 1.0
28.0 158.43173862310385 0.0
29.0 142.69601401664477 0.0
30.0 118.63636363636364 0.0
31.0 160.44044044044045 0.0
32.0 153.30666666666667 0.0
33.0 111.57045454545455 0.0
34.0 209.70154723127035 0.0
35.0 202.50298210735588 0.0
36.0 116.5 0.0
37.0 155.9645259938838 0.0
38.0 871.181560283688 1.0
39.0 168.56760

In [7]:
# does it work with the full images?

# paths
path_dapi = 'G:\\20241120_IOB_Magdalena\\20250520_TestClearing\\Organoid 1\\Full channels\\20250520_10x_Nq145um-C1.tif'
path_gfp = 'G:\\20241120_IOB_Magdalena\\20250520_TestClearing\\Organoid 1\\Full channels\\20250520_10x_Nq145um-C2.tif'
path_cy5 = 'G:\\20241120_IOB_Magdalena\\20250520_TestClearing\\Organoid 1\\Full channels\\20250520_10x_Nq145um-C3.tif'
path_gfp_mask = 'G:\\20241120_IOB_Magdalena\\20250520_TestClearing\\Organoid 1\\Full channels\\20250520_10x_Nq145um-C2__cp-mask.tif'
path_cy5_mask = 'G:\\20241120_IOB_Magdalena\\20250520_TestClearing\\Organoid 1\\Full channels\\20250520_10x_Nq145um-C3__cp-mask.tif'

# load images and masks as BioImage objects
dapi = io_utils.read_image(path_dapi)
gfp = io_utils.read_image(path_gfp)
cy5 = io_utils.read_image(path_cy5)
gfp_mask = io_utils.read_labels(path_gfp_mask)
cy5_mask = io_utils.read_labels(path_cy5_mask)

cy5_pos_gfp_mask = measure.overlapping_labels(cy5_mask, gfp_mask)

In [8]:
visualise.add_image(img=dapi.get_data(), name='dapi', colormap='blue') # scale = dapi.physical_pixel_sizes
visualise.add_image(img=gfp.get_data(), name='gfp', colormap='green')
visualise.add_image(img=cy5.get_data(), name='640', colormap='red')
visualise.add_labels(img=gfp_mask, name='gfp_mask') #, scale=dapi.physical_pixel_sizes)
visualise.add_labels(img=cy5_mask, name='cy5_mask')

visualise.add_labels(img=cy5_pos_gfp_mask, name="gfp_positive_Cy5-Objects")

In [10]:
import iob_ia.utils.filter_labels as fl

In [15]:
props_table = fl.measure_label_props(gfp_mask, gfp.get_data(), voxel_size=gfp.physical_pixel_sizes)

Measured properties of 23110 labels.
area: mean = 285.4125379810825; median = 32.46914772839942
euler_number: mean = 2.392038078753786; median = 2.0
extent: mean = 0.1621510491815661; median = 0.11904761904761904
intensity_mean: mean = 325.62260261250844; median = 286.5669642857143
intensity_min: mean = 210.2572046733016; median = 193.0
intensity_max: mean = 475.3987451319775; median = 386.0
projected_area: mean = 18.070343133876662; median = 7.182505027752638
projected_convex_area: mean = 24.584069134418726; median = 10.56250739375388
projected_perimeter: mean = 13.46989130098346; median = 9.23462263988918
projected_circularity: mean = 0.5867783963479958; median = 0.5703345804863066


In [24]:
props_tabel_mod, labels_to_remove = fl.filter_by_property(props_table, "intensity_mean")

Estimated minimal threshold value: 499.6541255498015
Filtering on intensity_mean between 499.6541255498015 and inf
Removed 20870 labels.Remaining labels: 2240 (previously 23110


({'label': {0: 1,
   2: 3,
   10: 11,
   37: 38,
   38: 39,
   75: 76,
   103: 104,
   105: 106,
   126: 127,
   127: 128,
   129: 130,
   130: 131,
   131: 132,
   133: 134,
   134: 135,
   135: 136,
   141: 142,
   151: 152,
   156: 157,
   160: 161,
   169: 170,
   170: 171,
   175: 176,
   183: 184,
   188: 189,
   190: 191,
   210: 211,
   219: 220,
   237: 238,
   238: 239,
   240: 241,
   243: 244,
   247: 248,
   251: 252,
   260: 261,
   264: 265,
   266: 267,
   268: 269,
   273: 274,
   276: 277,
   288: 289,
   290: 291,
   291: 292,
   293: 294,
   294: 295,
   303: 304,
   306: 307,
   307: 308,
   321: 322,
   326: 327,
   329: 330,
   330: 331,
   335: 336,
   336: 337,
   342: 343,
   347: 348,
   348: 349,
   352: 353,
   364: 365,
   365: 366,
   366: 367,
   368: 369,
   372: 373,
   375: 376,
   379: 380,
   392: 393,
   398: 399,
   399: 400,
   404: 405,
   405: 406,
   407: 408,
   424: 425,
   432: 433,
   443: 444,
   444: 445,
   446: 447,
   463: 464,
   480

In [30]:
newLabels, props_table_mod = fl.filter_labels_by_property(
    gfp_mask, props_table,
    {'intensity_mean': [0, float('inf')],
     'intensity_min': [None, None]}
)

Filtering on intensity_mean between 0 and inf
Removed 0 labels. Remaining labels: 23110 (previously 23110)
Estimated minimal threshold value: 263.7311414639869
Filtering on intensity_min between 263.7311414639869 and inf
Removed 18357 labels. Remaining labels: 4753 (previously 23110)
